## Utilizing Convolutional Neural Networks to Predict Emotions Through Visual Cues in Facial Recognition

In [ ]:
from sklearn.metrics import accuracy_score

In [3]:

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
from data_load import convert_data

In [ ]:

seed = 42
images, labels, classes, filepaths = convert_data()
Xtrain, Xval, Ytrain, Yval = train_test_split(images, labels, test_size=0.2, random_state=seed)
print(Xtrain.shape)
print(Xval.shape)
num_classes = len(Ytrain)


Current working directory: c:\Users\brand\OneDrive\Desktop\CS178\final_project\cs178_facial_expression
First few rows of the CSV:
  user.id                            image   emotion
0     628  facial-expressions_2868588k.jpg     anger
1     628  facial-expressions_2868585k.jpg  surprise
2     628  facial-expressions_2868584k.jpg   disgust
3     628  facial-expressions_2868582k.jpg      fear
4   dwdii           Aaron_Eckhart_0001.jpg   neutral

Sample filepaths in DataFrame:
  Image: facial-expressions_2868588k.jpg
  Filepath: c:\Users\brand\OneDrive\Desktop\CS178\final_project\cs178_facial_expression\images\facial-expressions_2868588k.jpg
  Image: facial-expressions_2868585k.jpg
  Filepath: c:\Users\brand\OneDrive\Desktop\CS178\final_project\cs178_facial_expression\images\facial-expressions_2868585k.jpg
  Image: facial-expressions_2868584k.jpg
  Filepath: c:\Users\brand\OneDrive\Desktop\CS178\final_project\cs178_facial_expression\images\facial-expressions_2868584k.jpg

Encoded emotion

In [5]:
from tensorflow.keras import layers, models

In [6]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(10000, 2500, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2,seed=seed))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.2,seed=seed*2))
model.add(layers.Dense(num_classes, activation='softmax'))

# build the CNN model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train the CNN model
history = model.fit(Xtrain, Ytrain, epochs=3, validation_data=(Xval, Yval))

# save the CNN model to saved_networks/tfmodel.keras
# model.save(save_path)

# evalulate the CNN model
result = model.evaluate(Xval, Yval)
result_dict = dict(zip(model.metrics_names, result))
print(f"Model Result\n-----\n{result_dict}")

c:\Users\brand\OneDrive\Desktop\CS178\final_project\cs178_facial_expression\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[99201024,128] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:Mul] name: 